# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

library와 framework의 차이는 제어 흐름에 대한 주도성이 누구에게 그리고 어디에 있는가에 있다. framework의 경우는 전체적인 흐름을 그 자체가 가지고 있어 사용자가 해당 framework 안에서 필요한 코드를 짜 넣는 구조이지만 library는 사용자가 그 흐름을 주도하여 필요한 library를 가져다가 사용한다.

### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

머신러닝은 인공지능의 한 분야로 경험을 통해 컴퓨터가 스스로 학습을 할 수 있게 해주는 알고리즘이다. 그리고 딥러닝은 마찬가지로 인공지능의 하위 개념이며 머신러닝에 속해있는 더 작은 개념이다. 딥러닝은 인공신경망 방법을 이용해 만든 머신러닝 기술이라고 할 수 있다.
이 두 개념의 차이는 사람의 개입 여부이다. 머신러닝은 특징추출과 같이 데이터를 사람이 먼저 처리한다. 그리고 부정확한 예측에 대해 엔지니어가 개입하여 조정한다. 반면에 딥러닝은 이러한 과정이 생략되어 신경망을 통해 컴퓨터가 데이터를 기반으로 온전히 스스로 학습하고 판단한다.

### Q3. 아래의 코드에 주석 달기.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

device = 'cuda' if torch.cuda.is_available() else 'cpu' #cuda를 사용 가능하다면 device 명을 cuda로 아니면 cpu로 설정
torch.manual_seed(45) #학습에서 random seed를 고정하기 위해 seed를 45로 제공함(cpu에서)
if device == 'cuda':
    torch.cuda.manual_seed_all(45) # 마찬가지로 seed를 모두 45로 고정하여 제공함.(cuda에서)
print(device + " is available") # 위에서 falser가 나왔으므로 cpu가 device 값으로 출력.

cpu is available


In [2]:
learning_rate = 0.001 # 학습률을 0.001로 설정
batch_size = 100 # 배치 사이즈는 100으로 설정
num_classes = 10 # 정답 클래스 개수를 10으로 설정
epochs = 5 # 에포크는 5로 설정

In [3]:
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',     # 데이터의 저장위치
    train = True,             # train_set으로 
    download = True,          # 다운로드 여부
    transform = transfroms.Compose([
        transfroms.ToTensor() # 데이터 전처리 작업(0에서 255까지 있는 값들을 0에서 1사이 값으로 변환)
    ])
)

test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',     # 데이터 저장위치
    train = False,            # test_set으로 (false)
    download = True,          # 다운로드 여부
    transform = transfroms.Compose([
        transfroms.ToTensor() # 데이터 전처리 작업(0에서 255까지 있는 값들을 0에서 1사이 값으로 변환)
    ])
)

# 훈련할 데이터셋인 MNIST(숫자 이미지)를 torchvision을 통해 불러옴.
# 해당 데이터들을 각각 train_set과 test_set으로 나눈 뒤 ToTensor를 통해 텐서로 변형해줌.

In [4]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

#앞선 dataset인 MNIST 데이터를 불러와 설정해준 batch_size 파라미터를 지정한 후 train_set과 test_set을 나눠 train_loader와 test_loader 생성.

examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

# train_set의 shape를 통해 input 데이터의 size를 확인할 수 있음. 이는 [1, 28(높이), 28(너비)]의 shape를 가지고 있음.

torch.Size([1, 28, 28])

In [7]:
class ConvNet(nn.Module):
  def __init__(self): # layer을 정의해주는 class를 생성
        super(ConvNet, self).__init__() 
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # input = 1, output filter = 10, kernel_size = 5, padding = 0(기본값), stride = 1(기본값)인 첫번째 2차원 배열의 convolutional layer 생성
                                                     # 이때 output size = ((28-5+2*0)/1)+1 = 24 즉 24x24로 변환시켜준다.
                                                     # max pooling을 통해 12x12로 변환
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # input = 10, output filter = 10, kernel_size = 5, padding = 0(기본값), stride = 1(기본값)인 두번째 2차원 배열의 convolutional layer 생성
                                                      # 이때 output size = ((12-5+2*0)/1)+1 = 8 즉 8x8로 변환시켜준다.
                                                      # max pooling을 통해 4x4로 변환            
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) # dropout 레이어를 추가하여 노드가 0이 될 확률을 0.5로 하 오버피팅 방지를 위해 사용.
        self.mp = nn.MaxPool2d(2) # 오버피팅을 방지하고 연산에 들어가는 자원을 줄이기 위해 사용(size가 2만큼 줄어들어 위의 max pooling 결과처럼 반으로 줄어들게 됨)
        self.fc1 = nn.Linear(320,100) # 4x4x20 vector을 100개의 출력으로 변경
        self.fc2 = nn.Linear(100,10)  # 100개의 출력을 10개의 출력으로 변경


  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) # 첫번째 convolution layer relu를 적용하고 maxpool, 결과값은 12x12x10
        x = F.relu(self.mp(self.conv2(x))) # 두번째 convolution layer relu를 적용하고 maxpool, 결과값은 12x12x10
        x = self.drop2D(x) 
        x = x.view(x.size(0), -1) # 전결합층을 위해 flatten
        x = self.fc1(x) # fc1 layer에 삽입
        x = self.fc2(x) # fc2 layer에 삽입
        return F.log_softmax(x) # fully-connected layer에 넣고 log softmax 함수에 적용해 리턴

In [9]:
model = ConvNet().to(device) # CNN 생성
criterion = nn.CrossEntropyLoss().to(device) # 비용함수
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # 옵티마이저(Adam 옵티마이저 사용, 모델의 매개변수와 학습률 하이퍼파라미터 등록)

In [10]:
for epoch in range(epochs): #epochs = 5 만큼 반복
    avg_cost = 0
    for data, target in train_loader:
        data = data.to(device) # 
        target = target.to(device)
        optimizer.zero_grad() # 모델의 gradient 값을 0으로 설정함.
        hypothesis = model(data) # 모델을 forward pass하여 결과값을 저장함.
        cost = criterion(hypothesis, target) # 저장한 output과 target의 loss를 계산하여 cost에 저장(비용함수)
        cost.backward() # backward 함수로 gradient 계산
        optimizer.step() # 모델 학습 파라미터 갱신
        avg_cost += cost / len(train_loader) # loss를 변수에 누적하여 train_loader 개수로 나누어줌(평균계산)
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

C:\Users\cts08\AppData\Local\Temp\ipykernel_22544\2022886463.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


[Epoch:    1]  cost = 0.338574588
[Epoch:    2]  cost = 0.114702247
[Epoch:    3]  cost = 0.0888829604
[Epoch:    4]  cost = 0.0762760714
[Epoch:    5]  cost = 0.0688434914


In [11]:
model.eval() # valuation 과정에서 사용하지 않아야 하는 layer을 off해줌. 즉, evaluate mode로 전환하여 dropout과 batch_normalization을 해제함.
with torch.no_grad(): # grad를 해제함.(graident 값을 저장하지 않음)
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] # 출력이 분류 각각에 대한 값으로 나타나기 때문에, 가장 높은 값을 갖는 인덱스를 추출
        total += len(target) # 전체 클래스 개수
        correct += (preds==target).sum().item() # 예측값과 실제값이 같은지 비교
        
    print('Test Accuracy: ', 100.*correct/total, '%')

C:\Users\cts08\AppData\Local\Temp\ipykernel_22544\2022886463.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Test Accuracy:  98.54 %


## 첫 정규세션 들으시느라 고생 많으셨습니다.